In [1]:
import os
import argparse
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm
import multiprocessing as mp
from pathlib import Path
import concurrent.futures
from collections import defaultdict
from multiprocessing import Pool, cpu_count
from networkx.convert_matrix import from_pandas_adjacency
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={"lines.linewidth": 2}, palette  = "deep", style = "ticks")
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from itertools import product, permutations, combinations, combinations_with_replacement

## Parameter interpretion
### tureEdgesDF
DataFrame of true network: columns('Gene1','Gene2'),386293 rows $\times$ 2 columns
### predEdgesDF
DataFrame of predicted network: columns('Gene1','Gene2','Edgeweight'),370254 rows $\times$ 3 columns
### possibleEdges
Set of all possible pairs: set(product(set(trueEdgesDF.Gene1),set(trueEdgesDF.Gene2)))
### TrueEdgeDict
Dictionary of true network: key:Gene1|Gene2 Value:0 or 1(represents whether key exits in tureEdgesDF)
### PredEdgeDict
Dictionary of predicted network: key:Gene1|Gene2 Value:max(np.abs(subDF.Edgeweight.values))

---

### possibleEdges_1
DataFrame of all possible pairs：the same as possibleEdges
### TrueEdgeDF_1
DataFrame of true network: columns('Gene1','Gene2','Value'), Value = 0 or 1(represents whether ('Gene1','Gene2') exits in tureEdgesDF)
### PredEdgeDF_2
DataFrame of predicted network: columns('Gene1','Gene2','Value') Value = max(np.abs(subDF.Edgeweight.values))

Read the gound truth DataFrame

In [2]:
trueEdgesDF = pd.read_csv('/gpfs/gibbs/pi/zhao/yw599/GRN/BEELINE-Networks/Networks/human/Non-specific-ChIP-seq-network.csv',sep = ',', header = 0, index_col = None)

In [3]:
trueEdgesDF

,Gene1,Gene2
0,AATF,BAX
1,AATF,CDKN1A
2,AATF,KLK3
3,AATF,MYC
4,AATF,TP53
...,...,...
386288,ZNF274,TNFRSF10B
386289,ZNF274,ZNF577
386290,ZNF384,PANK2
386291,ZNF740,MBD6


Initialize data dictionaries

In [4]:
precisionDict = {}
recallDict = {}
FPRDict = {}
TPRDict = {}
AUPRC = {}
AUROC = {}

Read the predicted GRN network

In [6]:
predEdgesDF = pd.read_csv('/gpfs/gibbs/pi/zhao/zc354/GRN/output/multiply_filename_unnorm.csv', sep = ',', header =  0, index_col = None)

In [7]:
predEdgesDF

,Gene1,Gene2,Edgeweight
0,ZNF98,LRAT,1.514776e-01
1,TCF4,PDZRN3,1.449918e-01
2,ZNF611,MTRNR2L12,1.302167e-01
3,KLF2,MTRNR2L12,1.203797e-01
4,POU2F2,PLD5,1.170610e-01
...,...,...,...
370249,EGR3,CPA3,3.588816e-12
370250,ZNF343,CPA3,3.584364e-12
370251,ZNF671,CPA3,3.527235e-12
370252,PRDM5,CPA3,3.404613e-12


In [8]:
true_TF = set(trueEdgesDF['Gene1'])
pred_TF = set(predEdgesDF['Gene1'])
true_tg = set(trueEdgesDF['Gene2'])
pred_tg = set(predEdgesDF['Gene2'])

In [9]:
TFset = true_TF & pred_TF
tgset = true_tg & pred_tg

In [10]:
trueEdgesDF = trueEdgesDF[(trueEdgesDF['Gene1'].isin(TFset))&(trueEdgesDF['Gene2'].isin(tgset))]
predEdgesDF = predEdgesDF[(predEdgesDF['Gene1'].isin(TFset))&(predEdgesDF['Gene2'].isin(tgset))]

## Function computeScores below<br />
Computes precision-recall and ROC curves using scikit-learn for a given set of predictions in the form of a DataFrame.

Generate all possible pairs

In [11]:
possibleEdges = set(product(set(trueEdgesDF.Gene1),set(trueEdgesDF.Gene2)))
#print(possibleEdges)
print(len(possibleEdges))
print(len(set(trueEdgesDF.Gene1)))
print(len(set(trueEdgesDF.Gene2)))

109224
222
492


### In this part, Keran used Dictionary to restore TrueEdge & PredEdge, and walk through each key to determine value:
 1 if edge is present in the ground-truth<br />
 0 if edge is not present in the ground-truth<br />
Creat the dictionary: '|' to sep Gene1 & Gene2, value=0

In [12]:
TrueEdgeDict = {'|'.join(p):0 for p in possibleEdges}
PredEdgeDict = {'|'.join(p):0 for p in possibleEdges}

In [13]:
print(len(TrueEdgeDict.keys()))
y=list(TrueEdgeDict.keys())
print(y[0],':',TrueEdgeDict[y[0]])

109224
TOPORS|TNFRSF1B : 0


In [12]:
import time

Compute TrueEdgeDict Dictionary<br />
1 if edge is present in the ground-truth<br />
0 if edge is not present in the ground-truth

In [14]:
#flag=0
#start_time=time.time()
for key in TrueEdgeDict.keys():

    if len(trueEdgesDF.loc[((trueEdgesDF['Gene1'] == key.split('|')[0]) &
                    (trueEdgesDF['Gene2'] == key.split('|')[1])) |
                        ((trueEdgesDF['Gene2'] == key.split('|')[0]) &
                    (trueEdgesDF['Gene1'] == key.split('|')[1]))]) > 0:
        TrueEdgeDict[key] = 1  
#    flag+=1
#    if flag==100:
#        break
#end_time=time.time()
#print(end_time-start_time)

For every 100 keys run for about 6.5 seconds<br />
For 50617700 keys, it will run for 3290150.5s=54835min=913h=38d

### In this part, try to use DataFrame to restoreTrueEdge & PredEdge, and walk through each row to determine value:
 1 if edge is present in the ground-truth<br />
 0 if edge is not present in the ground-truth<br />
use "apply" to speed up calculation

In [15]:
L=list(product(set(trueEdgesDF.Gene1),set(trueEdgesDF.Gene2)))
L1=[Gene[0] for Gene in L]
L2=[Gene[1] for Gene in L]
possibleEdges_1=pd.DataFrame({'Gene1':L1,'Gene2':L2})

In [16]:
possibleEdges_1

,Gene1,Gene2
0,HAND1,hsa-miR-4785
1,HAND1,PDIA2
2,HAND1,CASP6
3,HAND1,hsa-miR-542-5p
4,HAND1,PLS3
...,...,...
50617695,UBE2K,TUBGCP5
50617696,UBE2K,GRM8
50617697,UBE2K,NPHS1
50617698,UBE2K,PCDHGB6


In [17]:
TrueEdgeDF_1=possibleEdges_1
PredEdgeDF_1=possibleEdges_1

In [18]:
def TrueEdgeValue(row):
    gene1=row['Gene1']
    gene2=row['Gene2']
    if len(trueEdgesDF.loc[((trueEdgesDF['Gene1'] == gene1) &
                (trueEdgesDF['Gene2'] == gene2)) |
                    ((trueEdgesDF['Gene2'] == gene1) &
                (trueEdgesDF['Gene1'] == gene2))]) > 0:
        return 1
    else:
        return 0           

In [28]:
test=possibleEdges_1.iloc[:100]
test=test.copy()
#start_time=time.time()
test['Value'] = test.apply(lambda row:TrueEdgeValue(row),axis=1)
#end_time=time.time()
#print(end_time-start_time)

6.319425582885742


In [ ]:
#start_time=time.time()
with mp.Pool(mp.cpu_count()) as pool:
    test['Value'] = pool.map(TrueEdgeValue, test.iloc[:,:1])
#end_time=time.time()
#print(end_time-start_time)

In [33]:
from pandarallel import pandarallel

In [35]:
#start_time=time.time()
test['Value'] = test.parallel_apply(lambda row:TrueEdgeValue(row),axis=1)
#end_time=time.time()
#print(end_time-start_time)

AttributeError: 'DataFrame' object has no attribute 'parallel_apply'

In [21]:
test

,Gene1,Gene2,Value
0,HAND1,hsa-miR-4785,0
1,HAND1,PDIA2,0
2,HAND1,CASP6,0
3,HAND1,hsa-miR-542-5p,0
4,HAND1,PLS3,0
...,...,...,...
995,HAND1,MIR5681B,0
996,HAND1,MIR1243,0
997,HAND1,KLF5,0
998,HAND1,BCORL1,0


In [22]:
(test['Value']==1).sum()

0

In [ ]:
TrueEdgeDF_1=TrueEdgeDF_1.copy()
TrueEdgeDF_1['Value'] = TrueEdgeDF_1.apply(lambda row:TrueEdgeValue(row),axis=1)

### In this part, Keran used Dictionary to restore PredEdge, and walk through each key to determine value:
 1 if edge is present in the ground-truth<br />
 0 if edge is not present in the ground-truth<br />

In [38]:
#flag=0
#start_time=time.time()
for key in PredEdgeDict.keys():
    subDF = predEdgesDF.loc[((predEdgesDF['Gene1'] == key.split('|')[0]) &
                         (predEdgesDF['Gene2'] == key.split('|')[1])) |
                        ((predEdgesDF['Gene2'] == key.split('|')[0]) &
                        (predEdgesDF['Gene1'] == key.split('|')[1]))]
    if len(subDF)>0:
        PredEdgeDict[key] = max(np.abs(subDF.Edgeweight.values))
    #flag+=1
    #if flag==100:
        #break
#end_time=time.time()
#print(end_time-start_time)

6.386097431182861


In [25]:
def PredEdgesValue(row):
    gene1=row['Gene1']
    gene2=row['Gene2']
    subDF = predEdgesDF.loc[((predEdgesDF['Gene1'] == gene1) &
                         (predEdgesDF['Gene2'] == gene2)) |
                        ((predEdgesDF['Gene2'] == gene1) &
                        (predEdgesDF['Gene1'] == gene2))]
    if len(subDF)>0: 
        return max(np.abs(subDF.Edgeweight.values))
    else:
        return 0

In [26]:
test_2=possibleEdges_1.iloc[:100]
test_2=test_2.copy()
start_time=time.time()
test_2['Value'] = test_2.apply(lambda row:PredEdgesValue(row),axis=1)
end_time=time.time()
print(end_time-start_time)

KeyboardInterrupt: 

In [51]:
test_2

,Gene1,Gene2,Value
0,TRIM33,RG9MTD2,0
1,TRIM33,LOC652276,0
2,TRIM33,LOC100505659,0
3,TRIM33,SSX2B,0
4,TRIM33,RAB6A,0
...,...,...,...
4995,TRIM33,PPIA,0
4996,TRIM33,RND2,0
4997,TRIM33,C12orf65,0
4998,TRIM33,MN1,0


In [52]:
(test_2['Value']!=0).sum()

0

In [ ]:
# Combine into one dataframe
# to pass it to sklearn
outDF = pd.DataFrame([TrueEdgeDict,PredEdgeDict]).T
outDF.columns = ['TrueEdges','PredEdges']

prec, recall, thresholds = precision_recall_curve(y_true=outDF['TrueEdges'],
                                                      probas_pred=outDF['PredEdges'], pos_label=1)
precisionDict=prec
recallDict=recall
AUPRC=auc(recall, prec)

In [ ]:
PRName = '/uPRplot'
ROCName = '/uROCplot'
outDir='/gpfs/gibbs/pi/zhao/zc354/GRN/output/TF_filename_unnomalization_AUPRC'
##Make PR curves
legendList = []
for key in recallDict.keys():
    sns.lineplot(recallDict[key],precisionDict[key], ci=None)
    legendList.append(key + ' (AUPRC = ' + str("%.2f" % (AUPRC[key]))+')')
    plt.xlim(0,1)    
    plt.ylim(0,1)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend(legendList) 
    plt.savefig(outDir+PRName+'.pdf')
    plt.savefig(outDir+PRName+'.png')
    plt.clf()